In [1]:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import models, transforms

data_dir = "D:\\Trade-Off_CBIR\\dataset\\Corel-1K"
data_dir_GHIM = "D:\\Trade-Off_CBIR\\dataset\\GHIM-10K"

print("Classes available (Corel-1K):", os.listdir(data_dir))
print("Classes available (GHIM-10K):", os.listdir(data_dir_GHIM))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Classes available (Corel-1K): ['africans', 'beaches', 'buildings', 'buses', 'dinosaurs', 'elephants', 'flowers', 'food', 'horses', 'mountains']
Classes available (GHIM-10K): ['aircraft', 'buildings', 'butterfly', 'cars', 'dragon fly', 'fireworks', 'flowers', 'hen', 'horses', 'insects', 'motorcycles', 'mountains', 'sea shores', 'ships', 'sunset', 'temples', 'trees', 'valleys', 'walls', 'yacht']
Using device: cuda


## Tahap 1: Ekstraksi Fitur Menggunakan ConvNeXt

In [2]:
# CNN Models
efficientnet_v2 = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT).to(device).eval()
mobilenet_v3 = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.DEFAULT).to(device).eval()
resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT).to(device).eval()

# Transformasi standar untuk semua model (224x224)
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

## Tahap 2: Pre-trained Models

In [10]:
from tqdm import tqdm
import torch
import numpy as np
from PIL import Image
import os
from sklearn.preprocessing import normalize  # untuk L2-normalisasi

img_paths = []
labels = []

for cls in os.listdir(data_dir):
    cls_path = os.path.join(data_dir, cls)
    for f in os.listdir(cls_path):
        img_paths.append(os.path.join(cls_path, f))
        labels.append(cls)

# ===============================
# Ekstraksi fitur pre-trained CNN (pre-classifier) + normalisasi
# ===============================
efficientnet_features = []
mobilenet_features = []
resnet50_features = []

for path in tqdm(img_paths, desc="Extracting CNN Features"):
    img = Image.open(path).convert('RGB')
    img_t = transform(img).unsqueeze(0).to(device)  # type: ignore

    with torch.no_grad():
        # ---------- EfficientNet V2 Small ----------
        ef_x = efficientnet_v2.features(img_t)
        ef_x = torch.nn.functional.adaptive_avg_pool2d(ef_x, 1)
        ef_x = torch.flatten(ef_x, 1)
        efficientnet_features.append(ef_x.squeeze().cpu().numpy())

        # ---------- MobileNetV3 Small ----------
        mb_x = mobilenet_v3.features(img_t)
        mb_x = torch.nn.functional.adaptive_avg_pool2d(mb_x, 1)
        mb_x = torch.flatten(mb_x, 1)
        mobilenet_features.append(mb_x.squeeze().cpu().numpy())

        # ---------- ResNet50 ----------
        r50_x = resnet50.conv1(img_t)
        r50_x = resnet50.bn1(r50_x)
        r50_x = resnet50.relu(r50_x)
        r50_x = resnet50.maxpool(r50_x)
        r50_x = resnet50.layer1(r50_x)
        r50_x = resnet50.layer2(r50_x)
        r50_x = resnet50.layer3(r50_x)
        r50_x = resnet50.layer4(r50_x)
        r50_x = resnet50.avgpool(r50_x)
        r50_x = torch.flatten(r50_x, 1)
        resnet50_features.append(r50_x.squeeze().cpu().numpy())

# Convert ke numpy array
efficientnet_features = np.array(efficientnet_features)
mobilenet_features = np.array(mobilenet_features)
resnet50_features = np.array(resnet50_features)

# ===============================
# L2-normalisasi fitur
# ===============================
efficientnet_features = normalize(efficientnet_features)
mobilenet_features = normalize(mobilenet_features)
resnet50_features = normalize(resnet50_features)

Extracting CNN Features: 100%|██████████| 1000/1000 [00:45<00:00, 22.05it/s]


## Tahap 3: Image Retrieval & Similarity Matching

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# ===============================
# Fungsi retrieve top-K berdasarkan cosine similarity
# ===============================
def retrieve(query_feat, all_feats, top_k=10):
    sims = cosine_similarity([query_feat], all_feats)[0]  # type: ignore
    idxs = np.argsort(sims)[::-1][:top_k]
    return idxs

# ===============================
# Fungsi evaluasi Recall@K & Precision@K per kelas
# ===============================
def evaluate_recall_precision(features, labels, top_k=10):
    classes = sorted(list(set(labels)))
    results = []

    for cls in classes:
        # indeks query untuk kelas tertentu
        idxs_cls = [i for i, l in enumerate(labels) if l == cls]
        R_list, P_list = [], []

        for q in idxs_cls:
            retrieved_idx = retrieve(features[q], features, top_k)
            retrieved_labels = [labels[i] for i in retrieved_idx]

            # Recall = jumlah retrieved benar / total data kelas
            R = retrieved_labels.count(cls) / len(idxs_cls)
            # Precision = jumlah retrieved benar / top_k
            P = retrieved_labels.count(cls) / top_k

            R_list.append(R)
            P_list.append(P)

        results.append([cls, np.mean(R_list), np.mean(P_list)])

    return pd.DataFrame(results, columns=['Class','R','P'])


In [12]:
# ===============================
# Evaluasi CBIR / Retrieval hanya menggunakan model CNN
# ===============================
df_effnet = evaluate_recall_precision(efficientnet_features, labels)
df_mobilenet = evaluate_recall_precision(mobilenet_features, labels)
df_resnet50 = evaluate_recall_precision(resnet50_features, labels)

# Membuat tabel per kelas
df_table = pd.DataFrame()
df_table['Class'] = sorted(list(set(labels)))  # daftar kelas unik

df_table['EfficientNet V2 R'] = df_effnet['R']
df_table['EfficientNet V2 P'] = df_effnet['P']

df_table['MobileNetV3 R'] = df_mobilenet['R']
df_table['MobileNetV3 P'] = df_mobilenet['P']

df_table['ResNet50 R'] = df_resnet50['R']
df_table['ResNet50 P'] = df_resnet50['P']

# MEAN per kolom
mean_row = {
    'Class': 'Mean',
    'EfficientNet V2 R': df_effnet['R'].mean(),
    'EfficientNet V2 P': df_effnet['P'].mean(),
    'MobileNetV3 R': df_mobilenet['R'].mean(),
    'MobileNetV3 P': df_mobilenet['P'].mean(),
    'ResNet50 R': df_resnet50['R'].mean(),
    'ResNet50 P': df_resnet50['P'].mean(),
}

df_table = pd.concat([df_table, pd.DataFrame([mean_row])], ignore_index=True)

# Format angka 4 desimal
for col in df_table.columns[1:]:
    df_table[col] = df_table[col].astype(float).map("{:.4f}".format)

# Tampilkan tabel
display(df_table)

,Class,EfficientNet V2 R,EfficientNet V2 P,MobileNetV3 R,MobileNetV3 P,ResNet50 R,ResNet50 P
0,africans,0.0878,0.8780,0.0857,0.8570,0.0915,0.9150
1,beaches,0.0967,0.9670,0.0857,0.8570,0.0934,0.9340
2,buildings,0.0956,0.9560,0.0901,0.9010,0.0961,0.9610
3,buses,0.1000,1.0000,0.1000,1.0000,0.1000,1.0000
4,dinosaurs,0.1000,1.0000,0.1000,1.0000,0.1000,1.0000
5,elephants,0.1000,1.0000,0.0997,0.9970,0.1000,1.0000
6,flowers,0.1000,1.0000,0.1000,1.0000,0.1000,1.0000
7,food,0.0991,0.9910,0.0980,0.9800,0.0996,0.9960
8,horses,0.0999,0.9990,0.1000,1.0000,0.1000,1.0000
9,mountains,0.0994,0.9940,0.0920,0.9200,0.0983,0.9830


In [ ]:
import matplotlib.pyplot as plt

def average_precision_vs_topk(features_list, labels, top_k_max=100):
    top_ks = list(range(10, top_k_max + 1, 10))
    avg_prec = {name: [] for name in features_list}

    # Hitung Average Precision untuk setiap Top-K
    for top_k in top_ks:
        for name, feats in features_list.items():
            df = evaluate_recall_precision(feats, labels, top_k=top_k)
            avg_prec[name].append(df['P'].mean())

    plt.figure(figsize=(10, 6))

    # Marker dan warna per model
    markers = {
        'EfficientNet V2': 'o',
        'MobileNetV3': 's',
        'ResNet50': '^'
    }

    colors = {
        'EfficientNet V2': 'red',
        'MobileNetV3': 'blue',
        'ResNet50': 'green'
    }

    # Plot Average Precision vs Top-K
    for name in features_list:
        plt.plot(top_ks, avg_prec[name],
                 marker=markers.get(name, 'o'),
                 color=colors.get(name, 'black'),
                 linestyle='-',
                 linewidth=2,
                 markersize=7,
                 label=name)

    plt.xlabel("Number of Images Retrieved (Top-K)", fontsize=12)
    plt.ylabel("Average Precision", fontsize=12)
    plt.title("Average Precision vs Top-K for Corel-1K Dataset", fontsize=14)
    plt.xticks(top_ks)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(fontsize=12)
    plt.ylim(0, 1.05)
    plt.tight_layout()
    plt.show()

# Dictionary fitur baru
features_dict = {
    'EfficientNet V2': efficientnet_features,
    'MobileNetV3': mobilenet_features,
    'ResNet50': resnet50_features
}

# Jalankan evaluasi dan plot
average_precision_vs_topk(features_dict, labels, top_k_max=100)
